In [117]:
from dotenv import load_dotenv

load_dotenv()

True

# DYNAMO

In [118]:
from botocore.config import Config
from mypy_boto3_dynamodb.service_resource import DynamoDBServiceResource, Table
import boto3
from infrastructure.config.app_settings import get_app_settings

def _get_configuration() -> Table:
    app_settings=get_app_settings()
    _cfg = Config(
        retries={"max_attempts": 10, "mode": "standard"},
        connect_timeout=3,
        read_timeout=5
    )
    dynamo_resource: DynamoDBServiceResource = boto3.resource(
        "dynamodb",
        config=_cfg,
        region_name=app_settings.aws_settings.region
    )

    return dynamo_resource.Table(app_settings.table_settings.si_table)


In [47]:
table:Table=_get_configuration()

In [119]:
from typing import Any

def get_collection() -> list[dict[str, Any]]:
    items: list[dict[str, Any]] = []
    kwargs: dict[str, Any] = {
        "Limit": 100,                          # tamaño de página (opcional)
        # "ProjectionExpression": "#id, metadata",            # opcional: solo lo necesario
        # "ExpressionAttributeNames": {"#id": "id"},
        # "ConsistentRead": True,                            # solo tabla (no GSI)
    }

    while True:
        resp = table.scan(**kwargs)
        items.extend(resp.get("Items", []))
        lek = resp.get("LastEvaluatedKey")
        if not lek:
            break
        kwargs["ExclusiveStartKey"] = lek     # <-- siguiente página

    return items


In [126]:
results=get_collection()
results_tasaciones=[d for d in results if d.get("metadata",{}).get("document_type",None) == "tasaciones"]
print(len(results))
print(len(results_tasaciones))

#print("results:", results_tasaciones)

326
155


In [127]:
result_id=results_tasaciones[0].get("id")
result_to_search=results_tasaciones[0].get("metadata",{})
content_llm=result_to_search.get("document_content_llm","")
print(content_llm)


964331474-964768205
SUPERITENDENCIA DE BANCA Y SEGUROS
CAP. N° 2287
Calle 13 de Julio S/N, Mz. M, Lote 41
RESUMEN DE TASACION PARA USO EXCLUSIVO DEL
Carhuamayo, 02 de Diciembre del 2019
Partida Registral : P16024370
UBICACIÓN REGISTRAL
JORGE GABRIEL SOLOGUREN TENICELA
REFEV ARQUITECTO PERINTENDENCIA DE BANCAY SEGUROS N°
79,366.00
UBICACIÓN MUNICIPAL
CMAC HUANCAYO S.A.
C.A.P. 2287 PERITO 2344-2011
Valor Nuevo
Distrito: San Agustin de ajas, Provincia: Huancayo, Departamento: Junin
VALUACION DE INMUEBLE URBANO A VALOR COMERCIAL
$
63,492.80
PROPIETARIOS
(VALORIZACIÓN)
Telf 064 -335705 964956439
$
265,082.50
S/.
Valor de Realización Inmediato
212,066.00
Distrito: San Agustin de ajas, Provincia: Huancayo, Departamento: Junin
Sector Pampa del Carmen Chanchamayo
JORGE GABRIEL SOLOGUREN TENICELA VALUADOR
3.34
OFICINA. Calle Las Pitucas Nº 181
UBICACIÓN IN SITU
CORREA ATAHUAMAN MARCELINO FORTUNATO
S/.
S/.
Centro Poblado San Agustin de Cajas, Sector 2, Mz. M, Lote N° 41
SOLICITANTE
Distrito: San 

In [109]:
print("id", result_id)

id b8b7acd2-876a-4b76-8b0c-a605bfadccf6


# Bedrock

In [ ]:
from pydantic import SecretStr
from langchain_aws import ChatBedrockConverse
from infrastructure.config.app_settings import AppSettings

app_settings: AppSettings = get_app_settings()

def get_bedrock() -> ChatBedrockConverse:
    retries_config: Any = {"max_attempts": 12, "mode": "adaptive"}
    config = Config(
        retries=retries_config,
        read_timeout=300,
        connect_timeout=10,
        tcp_keepalive=True,
        max_pool_connections=50
        )
    client = boto3.client("bedrock-runtime", region_name=app_settings.aws_settings.region,
                      config=config)
    return ChatBedrockConverse(
        model="anthropic.claude-3-5-sonnet-20240620-v1:0",
        region_name=app_settings.aws_settings.region,
        bedrock_client=client,
    )
bedrock_converse: ChatBedrockConverse = get_bedrock()


In [129]:
from domain.models.states.etl_tasaciones_state import EtlTasacionesState
from infrastructure.adapters.transformers.bed_rock_transformer_document import BedRockTransformerDocument

def llm_tasaciones_internal_chain(context: str) -> EtlTasacionesState:
    tasacion_system_prompt = """Eres un experto obteniendo información de las tasaciones, para lo cual debes
    obtener información del nombre del perito (el cual por lo general lo encuentra como perito evaluador o perito
    y tambien sobre palabras de ing. o lic), la fecha de la tasación (por lo general cerca a la palabra fecha);
    el valor comercial en soles (puede estar alrededor de palabras como Valor comercial (VC) SOLES S/.) y el
    valor de realización en soles igual puede estar alrededor de palabras como Valor de realización (VR) SOLES
    S/; también debes obtener al propietario de la tasación por lo general esta alrededor de la palabra
    "propietario" o "propietaria" tus hallazgos los debes retornar en json de la siguiente forma: {
    "expert_warranty_name": "Indica el nombre del perito", "tasacion_date": "Indica la fecha de la tasación, retornalo en formato dd/mm/aaaa",
    "commercial_value": "Indica el valor comercial en soles ( PEN)", "realization_value": "Indica el valor de
    realización en soles (PEN)", "tasacion_owner": "Indica el nombre del propietario de la tasación" } ;"""
    messages = [
        ("system", f"{tasacion_system_prompt}"),
        ("human", f"{context}")
    ]
    chain = bedrock_converse.with_structured_output(EtlTasacionesState)
    results = chain.invoke(messages)
    return EtlTasacionesState.model_validate(results)

def llm_caller_tasaciones(context: str) -> EtlTasacionesState | None:
        return BedRockTransformerDocument.with_throttling_retry(llm_tasaciones_internal_chain, context)



In [130]:
result_llm =llm_caller_tasaciones(content_llm)

In [131]:
print("llm", result_llm)

llm record_id='964331474-964768205' document_content_total=None document_content_llm=None period_month=None period_year=None extract_success=None transform_success=None load_success=None expert_warranty_name='JORGE GABRIEL SOLOGUREN TENICELA' tasacion_date='02/12/2019' commercial_value='265082.50' realization_value='212066.00' tasacion_owner='CORREA ATAHUAMAN MARCELINO FORTUNATO'


# Salvar metadata

In [132]:
all_data=results_tasaciones[0]
metadata=all_data.get("metadata",{})
metadata={**metadata, "expert_warranty_name":result_llm.expert_warranty_name, "tasacion_date":result_llm.tasacion_date}

to_save={
    **all_data,
    "metadata":metadata,
}


In [137]:
def save_metadata(data:dict[str,Any])->None:
    print("guardando en dynamo")
    table.put_item(Item=data)


In [123]:
save_metadata(to_save)

# Pipeline de actualizacion

In [142]:
def save_data():
    for element in results_tasaciones:
        new_content_llm=element.get("metadata",{}).get("document_content_llm","")
        new_result_llm=llm_caller_tasaciones(new_content_llm)
        metadata=element.get("metadata",{})
        metadata={**metadata,"expert_warranty_name":new_result_llm.expert_warranty_name, "tasacion_date":new_result_llm.tasacion_date}
        print("id",element.get("id"))
        print("nueva metadata", metadata["tasacion_date"],metadata["expert_warranty_name"])
        to_save={
            **element,
            "metadata":metadata,
        }
        save_metadata(to_save)


In [143]:
save_data()

id b8b7acd2-876a-4b76-8b0c-a605bfadccf6
nueva metadata 02/12/2019 JORGE GABRIEL SOLOGUREN TENICELA
guardando en dynamo
[Retry] Throttling detectado. Esperando 1.15s antes de reintentar (1/5)
[Retry] Throttling detectado. Esperando 2.51s antes de reintentar (2/5)
[Retry] Throttling detectado. Esperando 4.77s antes de reintentar (3/5)
[Retry] Throttling detectado. Esperando 8.51s antes de reintentar (4/5)
id 831f1af9-caa9-46b0-b508-71ea40d296d9
nueva metadata 30/05/2023 Ing. Roy Lapa Taipe
guardando en dynamo
[Retry] Throttling detectado. Esperando 1.60s antes de reintentar (1/5)
[Retry] Throttling detectado. Esperando 2.85s antes de reintentar (2/5)
[Retry] Throttling detectado. Esperando 4.14s antes de reintentar (3/5)
[Retry] Throttling detectado. Esperando 8.56s antes de reintentar (4/5)
[Retry] Throttling detectado. Esperando 16.68s antes de reintentar (5/5)
id 559d9030-c3bd-4615-9c1b-4680567aec18
nueva metadata 12/02/2020 Ing. GIANNINA CARMEN TAZA PIÑAS
guardando en dynamo
[Retry] 